In [ ]:
import cv2
import numpy
import depthai as dai
%matplotlib inline
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Image

MAX_DEPTH = 1500.0  # unit: [mm]

pipeline = dai.Pipeline()

cam_left = pipeline.create(dai.node.MonoCamera)
cam_left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_480_P)
cam_right = pipeline.create(dai.node.MonoCamera)
cam_right.setResolution(dai.MonoCameraProperties.SensorResolution.THE_480_P)
stereo = pipeline.create(dai.node.StereoDepth)
xout_depth = pipeline.create(dai.node.XLinkOut)
xout_depth.setStreamName("depth")

cam_left.out.link(stereo.left)
cam_right.out.link(stereo.right)
stereo.depth.link(xout_depth.input)

display_handle=display(None, display_id=True)
while(True):
    with dai.Device(pipeline) as device:
        queueDepth = device.getOutputQueue(name="depth", maxSize=4, blocking=False)
        while True:
            inDepth = queueDepth.tryGet()
            depth = None
            if inDepth is not None:
                depth = inDepth.getCvFrame()
            if depth is not None:
                depth[numpy.where(depth > MAX_DEPTH)] = 0
                norm_depth = (depth * (255 / MAX_DEPTH)).astype(numpy.uint8)
                colored_depth: numpy.ndarray = cv2.applyColorMap(norm_depth, cv2.COLORMAP_JET)
                rows, columns, _ = colored_depth.shape
                colored_depth = cv2.resize(colored_depth, (int(columns/2), int(rows/2)))
                _, jpg = cv2.imencode('.jpeg', colored_depth)
                display_handle.update(Image(data=jpg.tobytes()))